In [1]:
from DrissionPage import ChromiumPage
from DataRecorder import Recorder
from datetime import date
from tqdm import tqdm
import time
import json

def sign_in():
    """
    首次运行需要登录小红书
    """
    sign_in_page = ChromiumPage()
    sign_in_page.get('https://www.xiaohongshu.com')
    print("请扫码登录")
    time.sleep(20)

def read_urls_from_txt(path):
    """
    从txt文件中读取所有笔记链接
    """
    with open(path, 'r') as file:
        urls = [line.strip() for line in file.readlines()]
    return urls

def open_url(url):
    """
    打开小红书笔记详情页
    """
    global page
    page = ChromiumPage()
    page.get(f'{url}')
    time.sleep(3)  # 等待页面加载
    return page

def get_meta_content(page, meta_name):
    """
    获取指定meta标签的内容
    """
    try:
        meta = page.ele(f'xpath://meta[@name="{meta_name}"]', timeout=5)
        return meta.attr('content') if meta else ''
    except:
        return ''

def get_note_info(page):
    """
    从meta标签获取笔记信息
    """
    try:
        # 获取标题
        title = get_meta_content(page, 'og:title')
        if title:
            title = title.replace(' - 小红书', '')  # 移除后缀
            
        # 获取描述（笔记内容）
        content = get_meta_content(page, 'description')
        
        # 获取链接
        note_link = get_meta_content(page, 'og:url')
        
        # 获取标签（关键词）
        keywords = get_meta_content(page, 'keywords')
        tags = keywords.split(', ') if keywords else []
        
        # 获取统计数据
        like_count = get_meta_content(page, 'og:xhs:note_like')
        collect_count = get_meta_content(page, 'og:xhs:note_collect')
        comment_count = get_meta_content(page, 'og:xhs:note_comment')
        
        info = {
            'title': title,
            'content': content,
            'note_link': note_link,
            'tags': ','.join(tags),
            'like_count': like_count,
            'collect_count': collect_count,
            'comment_count': comment_count
        }
        return info
    except Exception as e:
        print(f"获取笔记信息失败: {str(e)}")
        return {
            'title': '',
            'content': '',
            'note_link': '',
            'tags': '',
            'like_count': '0',
            'collect_count': '0',
            'comment_count': '0'
        }

def get_note_page_info(url):
    """
    获取笔记页面所有信息
    """
    # 访问url
    page = open_url(url)
    
    # 获取笔记信息
    note_info = get_note_info(page)
    
    return note_info

if __name__ == '__main__':
    try:
        # 第1次运行需要登录，后面不用登录，可以注释掉
        sign_in()

        # 获取当前日期
        current_date = date.today()
        
        # 创建输出文件名
        output_file = f'采集输出-小红书笔记详情-{current_date}.xlsx'

        # 新建一个excel表格，用来保存数据
        r = Recorder(path=output_file, cache_size=20)

        # 设置要采集的笔记链接
        note_urls_file_path = '需要采集的笔记链接（每行放1个链接）.txt'

        # 从txt文件读取urls
        note_urls = read_urls_from_txt(note_urls_file_path)

        for note_url in tqdm(note_urls):
            try:
                # 采集笔记详情
                note_info = get_note_page_info(note_url)
                
                # 整理数据格式
                new_note_contents_dict = {
                    '采集日期': str(current_date),
                    '作者': '',  # 暂时无法从meta标签获取作者信息
                    '笔记标题': note_info['title'],
                    '发布日期': '',  # 暂时无法从meta标签获取发布日期
                    'IP属地': '',  # 暂时无法从meta标签获取IP属地
                    '点赞数': note_info['like_count'],
                    '收藏数': note_info['collect_count'],
                    '评论数': note_info['comment_count'],
                    # '笔记链接': note_info['note_link'],
                    '笔记链接': note_url,
                    '作者链接': '',  # 暂时无法从meta标签获取作者链接
                    '标签': note_info['tags'],
                    '笔记内容': note_info['content']
                }
                
                # 数据写入缓存
                r.add_data(new_note_contents_dict)
                
                # 避免频繁请求
                time.sleep(3)
                
            except Exception as e:
                print(f"处理笔记 {note_url} 时出错: {str(e)}")
                continue

        # 保存excel文件
        r.record()
        print("数据采集完成！")
        
    except Exception as e:
        print(f"程序执行出错: {str(e)}")
        # 确保数据被保存
        try:
            r.record()
        except:
            pass

请扫码登录


100%|██████████| 3/3 [00:25<00:00,  8.44s/it]

采集输出-小红书笔记详情-2025-02-19.xlsx 开始写入文件，切勿关闭进程。
采集输出-小红书笔记详情-2025-02-19.xlsx 写入文件结束。
数据采集完成！
